## GOAL: Create a Regression Model that can predict house price based on features of that house

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Data

In [2]:
df = pd.read_csv("DATA/AMES_Final_DF.csv")

In [3]:
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


### The label we are trying to predict is the SalePrice column.

In [5]:
X = df.drop('SalePrice',axis=1)
y = df['SalePrice']

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=101)

### The dataset features has a variety of scales and units. For optimal regression performance, we scale the X features.

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()

In [10]:
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

### We will use an Elastic Net model.

In [11]:
from sklearn.linear_model import ElasticNet

In [12]:
base_elastic_model = ElasticNet()

### We Using scikit-learn create a GridSearchCV object and run a grid search for the best parameters for model based on the scaled training data.

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
param_grid = {
    'alpha': [0.1, 1, 5, 10, 50, 100],
    'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
}

In [15]:
grid_model = GridSearchCV(estimator=base_elastic_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

In [16]:
grid_model.fit(scaled_X_train,y_train)

C:\Users\Kazoon\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 139422008252.95306, tolerance: 1355206692.5276787
  model = cd_fast.enet_coordinate_descent(
C:\Users\Kazoon\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 165405536738.52106, tolerance: 1307913805.6588454
  model = cd_fast.enet_coordinate_descent(
C:\Users\Kazoon\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 132401459345.12183, tolerance: 1415056940.006106
  model = cd_fast.enet_coordinate_descent(
C:\Users\Kazoon\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Converge

C:\Users\Kazoon\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 212820336042.7881, tolerance: 1438198040.088288
  model = cd_fast.enet_coordinate_descent(
C:\Users\Kazoon\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4848819954.13147, tolerance: 1345680018.2551236
  model = cd_fast.enet_coordinate_descent(
C:\Users\Kazoon\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13130431879.618896, tolerance: 1355206692.5276787
  model = cd_fast.enet_coordinate_descent(
C:\Users\Kazoon\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergence

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error')

In [17]:
grid_model.best_params_

{'alpha': 100, 'l1_ratio': 1}

### Evaluate model performance on the unseen 10% scaled test set.

In [18]:
y_pred = grid_model.predict(scaled_X_test)

In [19]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [20]:
mean_absolute_error(y_test,y_pred)

14195.354900562166

In [21]:
np.sqrt(mean_squared_error(y_test,y_pred))

20558.508566893157

In [23]:
np.mean(df['SalePrice'])

180815.53743589742